![clothing_classification](clothing_classification.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.

In [1]:
# Run the cells below first

In [1]:
!pip install torchmetrics
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 60.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [3]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



Number of classes

In [21]:
classes = train_data.classes
num_classes = len(train_data.classes)

num_input_channels = 1
num_output_channels = 16
image_size = train_data[0][0].shape[1]

CNN

In [22]:
class MultiClassImageClassifier(nn.Module):
    def __init__(self, num_classes):
        super(MultiClassImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(num_input_channels, num_output_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

        self.fc = nn.Linear(num_output_channels * (image_size//2)**2, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

Training DataLoader

In [23]:
dataloader_train = DataLoader(
    train_data,
    batch_size=10,
    shuffle=True,
)

Training Function

In [24]:
def train_model(optimizer, net, num_epochs):
    num_processed = 0
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        running_loss = 0
        num_processed = 0
        for features, labels in dataloader_train:
            optimizer.zero_grad()
            outputs = net(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            num_processed += len(labels)
        print(f'epoch {epoch}, loss: {running_loss / num_processed}')
        
    train_loss = running_loss / len(dataloader_train)

Training for 1 epoch

In [25]:
net = MultiClassImageClassifier(num_classes)
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [26]:
train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=1,
)

epoch 0, loss: 0.03898408574059916


Test DataLoader

In [27]:
dataloader_test = DataLoader(
    test_data,
    batch_size = 10,
    shuffle = False
)

Metrics

In [28]:
accuracy_m = Accuracy(task='multiclass', num_classes=num_classes)
precision_m = Precision(task='multiclass', num_classes=num_classes, average=None)
recall_m = Recall(task='multiclass', num_classes=num_classes, average=None)

Compute metrics

In [29]:
net.eval()
predictions = []

for i, (features,labels) in enumerate(dataloader_test):
    output = net.forward(features.reshape(-1,1, image_size, image_size))
    cat = torch.argmax(output, dim=1)
    predictions.extend(cat.tolist())
    accuracy_m(cat, labels)
    precision_m(cat, labels)
    recall_m(cat, labels)
    
accuracy = accuracy_m.compute().item()
precision = precision_m.compute().tolist()
recall = recall_m.compute().tolist()

print('Accuracy:', accuracy)
print('Precision (per class):', precision)
print('Recall (per class):', recall)

Accuracy: 0.8881000280380249
Precision (per class): [0.8095238208770752, 0.9808080792427063, 0.8508108258247375, 0.8881118893623352, 0.7862939834594727, 0.9876288771629333, 0.7087486386299133, 0.9342995285987854, 0.9749498963356018, 0.9589178562164307]
Recall (per class): [0.8669999837875366, 0.9710000157356262, 0.7870000004768372, 0.8889999985694885, 0.871999979019165, 0.9580000042915344, 0.6399999856948853, 0.9670000076293945, 0.9729999899864197, 0.9570000171661377]
